## Importing necessary dependencies

In [2]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

### Loading document

In [4]:
import pandas as pd

chunk_size=100
chunks=pd.read_csv("C:\\Users\\srile\\Downloads\\aws_review_sofware_dataset.csv",sep=',',chunksize=chunk_size)
df=next(chunks)
print(df.columns)

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [5]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [6]:
df["words"]="default value"
df["sentences"]="default value"

for i in range(df.shape[0]):
    df.at[i,"words"]=list("")
    df.at[i,"sentences"]=list("")

###  Importing sent Tokenization from NLTK

In [7]:
from nltk.tokenize import sent_tokenize

In [8]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
for i in range(df.shape[0]):
    l1=sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

In [10]:
# pywsd library is a tool for resolving word ambiguity in NLP
!pip install pywsd

Defaulting to user installation because normal site-packages is not writeable


## Importing averaged_perceptron_tagger finding POS_tagging

In [11]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [16]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 11.31167197227478 secs.


In [17]:

# importing wordnet is used for meanings, synonyms, antonyms, and relationships to other words. 
# other NLP (Natural Language Processing) tasks.
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [19]:
df["words_sentences"]="default"


In [20]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])

## Importing CountVectorizer

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
df1=df
no_features=1000
tf_vectorizer=CountVectorizer(max_features=no_features,stop_words='english')
tf=tf_vectorizer.fit_transform(df1.words_sentences)


In [23]:
# df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())
# Convert sparse matrix to dense array
df_x = pd.DataFrame(tf.toarray(), columns=tf_vectorizer.get_feature_names_out())

In [24]:

df_y=df["verified"]

## Importing LabelEncoder

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le=LabelEncoder()

In [35]:
df_y_1=pd.DataFrame(df_y)

In [36]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [37]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [38]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


In [39]:
#Import Libraries:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Random Forest Initialization:
rf=RandomForestClassifier(n_estimators=500,random_state=42)

#Fit the Model:
rf.fit(df_x,df_y_enc)

accuracy_rf=rf.score(df_x,df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf*100:.2f}%")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 100.00%


In [40]:
#Import Library:
from sklearn.naive_bayes import MultinomialNB

#Initialize Classifier:
nb=MultinomialNB()

#Fit the Model
nb.fit(df_x,df_y_enc)

accuracy_nb=nb.score(df_x,df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb*100:.2f}%")

Naive Bayes Accuracy: 93.00%


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:

from sklearn.ensemble import GradientBoostingClassifier

In [42]:

# Initialize the Gradient Boosting Classifier
GBC=GradientBoostingClassifier(n_estimators=1000)

In [43]:
## Fit the model
gb_c=GBC.fit(df_x,df_y_enc)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
# Calculate accuracy
gbc_score=GBC.score(df_x,df_y_enc)
print(f"gbc_score: {gbc_score*100:.2f}%")


gbc_score: 100.00%
